In [31]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
def load_data(file_path):
    """
    Load the game data from a CSV file.
    """
    df = pd.read_csv(file_path)
    return df


# Problem 1
### For the first question, to group the various strategies used by the players, we learn the data for each game and player using RandomForestClassifier. We then use this model to check if this same model predicts the moves of some other player upto 82% accuracy. If this is the case then the code classifies the strategies used by those two players as the same strategy. This is the done for the whole dataset i.e. all of the games and the corresponding model for each strategy is stored. The featured used for training the RandomForestClassifier is the trust percentage of the opponent uptil that point and the previous 10 moves of both players. Finally the count of the distinct strategies is printed.

# Problem 2
### The list of players is obtained directly from the previous question as the list of players with same strategies is stored as and when they are found.

In [32]:
def feature_extraction(df):
    """
    Extract features from the game data.
    """
    df['p1_action'] = df['p1_action'].apply(lambda x: 1 if x == 'TRUST' else 0)
    df['p2_action'] = df['p2_action'].apply(lambda x: 1 if x == 'TRUST' else 0)

    # Calculate the cumulative sum of TRUST actions for each player and the cumulative number of turns
    df['p1_cum_trust'] = df.groupby('game_id')['p1_action'].cumsum()
    df['p2_cum_trust'] = df.groupby('game_id')['p2_action'].cumsum()
    df['cum_turns'] = df.groupby('game_id').cumcount() + 1  # +1 because cumcount starts from 0

    # Calculate the total percent of TRUST for each player up to that turn
    df['p1_trust_percent'] = df['p1_cum_trust'] / df['cum_turns']
    df['p2_trust_percent'] = df['p2_cum_trust'] / df['cum_turns']
    for i in range(1, 11):
        # Shift the actions to get the last i moves, filling missing values with a placeholder (-1)
        df[f'p1_move_{i}'] = df.groupby('game_id')['p1_action'].shift(i-1).fillna(-1).astype(int)
        df[f'p2_move_{i}'] = df.groupby('game_id')['p2_action'].shift(i-1).fillna(-1).astype(int)
        df_p1 = df.copy()
    df_p2 = df.copy()

    # Add 'player' and 'action' columns for p1 and p2 rows
    df_p1['player'] = df_p1['p1_id']
    df_p1['action'] = df_p1['p1_action']

    df_p2['player'] = df_p2['p2_id']
    df_p2['action'] = df_p2['p2_action']

    # Combine the two dataframes
    df_combined = pd.concat([df_p1, df_p2], ignore_index=True)

    # Sort the dataframe by 'player'
    df_combined_sorted = df_combined.sort_values(by=['player', 'game_id'])
    df_grouped = df_combined_sorted.groupby(['player', 'game_id']).apply(lambda x: x.iloc[10:]).reset_index(drop=True)

    # For each 'game_id' for each 'player', remove the first 10 columns for every 'game_id', retaining only 'player', 'game_id', and the new 'action'

    # Display the first few rows of the transformed dataframe to verify the changes
    ##print(df)


    return df_grouped

In [33]:
def model_train(X,y):
    feature_columns = ['p1_cum_trust', 'p2_cum_trust'] + [f'p1_move_{i}' for i in range(1, 11)] + [f'p2_move_{i}' for i in range(1, 11)]
    ##X = df.loc[10:57, feature_columns]  # Python uses zero-based indexing, so row 11 is at index 10, and we use 57 to include row 58
    ##y = df.loc[10:57, 'p1_action'].shift(-1).fillna(-1)  # Adjust y accordingly
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f'Accuracy: {accuracy:.2f}')
    return model

In [34]:
def group_by_strategy(df):
    """
    Group players by their identified strategies.
    """
    # This example assumes that the 'strategy' column has been filled with identified strategy names or IDs.
    strategy_groups = defaultdict(list)
    for _, row in df.iterrows():
        strategy_groups[row['strategy']].append(row['p1_id'])
        strategy_groups[row['strategy']].append(row['p2_id'])

    # Remove duplicate player IDs in each strategy group and sort
    for strategy, players in strategy_groups.items():
        strategy_groups[strategy] = sorted(list(set(players)))
    
    # Convert the strategy groups from a dictionary to a list of lists
    grouped_strategies = list(strategy_groups.values())
    
    # Sort the list of lists based on the smallest player ID in each group
    grouped_strategies.sort(key=lambda x: x[0])
    
    return grouped_strategies

In [35]:
def model_fit(df, model,num, total_num, player1,total_set):
    fit_counter = 0
    feature_columns = ['p1_cum_trust', 'p2_cum_trust'] + [f'p1_move_{i}' for i in range(1, 11)] + [f'p2_move_{i}' for i in range(1, 11)]
    set = []
    num+=1
    set.append(player1)
    total_set.append(player1)
    # Iterate through each group of games by player
    for player, group in df.groupby('player'):
        # Selecting features and labels
        game_X = group[feature_columns]  # Correct way to select features
        game_y = group['action']  # Assuming 'action' is the correct label column based on your transformation
        
        # Check if there are enough data points to predict and calculate accuracy
        if len(game_X) > 0:
            game_y_pred = model.predict(game_X)
            
            # Calculating accuracy for the current player's games
            accuracy = accuracy_score(game_y, game_y_pred)
            
            # Increment the counter if the accuracy meets a certain threshold, e.g., > 0.7
            if accuracy > 0.82:
                if(player not in total_set):
                    fit_counter += 1
                    num += 1
                    set.append(player)
                    total_set.append(player)

    return num, set, total_set

In [36]:
def main():
    # Path to your dataset
    file_path = 'input_game.csv'
    
    # Load the dataset
    df = load_data(file_path)
    df = feature_extraction(df)
    feature_columns = ['p1_cum_trust', 'p2_cum_trust'] + [f'p1_move_{i}' for i in range(1, 11)] + [f'p2_move_{i}' for i in range(1, 11)]
    total_set = []
    num = 0
    count = 0
    models = []
    final_out=[]
    total_num = pd.concat([df['p1_id'], df['p2_id']]).nunique()
    for player, group in df.groupby('player'):
        if player not in total_set:
            player_X = group[feature_columns]  # Features for the player across all their games
            player_y = group['action'].shift(-1).fillna(-1)  # Target for the player, shifting actions to predict the next action
            model = model_train(player_X, player_y)  # Train the model for the player
            num,set, total_set = model_fit(df, model,num,total_num,player,total_set)  # Fit the model for the player, assuming '1' is a parameter for model_fit
            if len(set) < 3:
                for player in set:
                    total_set.remove(player)
                continue
            #print(player)
            models.append(model)
            set.sort()
            #print(set)
            final_out.append(set)
            count+=1
            #print(num)
    final_out = sorted(final_out, key=lambda x: x[0])
    print(final_out)
    print(count)


In [37]:
# If running as a script, execute the main function
if __name__ == '__main__':
    main()


[[1, 2, 3, 4, 5, 7, 9, 13, 15, 19, 20, 25, 27, 33, 34, 36, 37, 39, 53, 55, 60, 65, 76, 87, 95, 100, 104, 105, 110, 118, 121, 126, 128, 129, 136, 137, 141, 143, 146, 149, 150, 158, 172, 175, 178, 181, 192, 193, 194, 200], [6, 8, 16, 24, 29, 30, 32, 41, 43, 47, 50, 52, 59, 61, 62, 66, 70, 73, 81, 84, 86, 88, 89, 91, 94, 101, 102, 107, 109, 116, 117, 120, 125, 133, 134, 135, 139, 142, 144, 145, 153, 156, 157, 162, 166, 170, 171, 174, 176, 183, 184, 185, 187, 195], [10, 83, 119, 124, 173], [14, 22, 23, 38, 57, 64, 85, 96, 97, 108, 111, 115, 131, 148, 155, 167, 189, 197], [42, 54, 58, 74, 75, 80, 93, 130, 147, 164, 179, 199], [45, 56, 67, 77, 79, 92, 103, 113, 138, 152, 159, 161, 168, 169, 180, 182, 188, 190]]
6


### This is the list of players with the same strategy and the total count is 6.

In [38]:
def simulate_game(strategy1, strategy2, num_rounds):
    """
    Simulates a game between two strategies for a fixed number of rounds.
    
    :param strategy1: The first player's strategy/model.
    :param strategy2: The second player's strategy/model.
    :param num_rounds: Number of rounds to play.
    :return: The total score for both strategies.
    """
    # Initialize scores
    score1, score2 = 0, 0
    
    # Reward matrix
    rewards = {
        ('Trust', 'Trust'): (2, 2),
        ('Cheat', 'Cheat'): (0, 0),
        ('Trust', 'Cheat'): (-1, 3),
        ('Cheat', 'Trust'): (3, -1),
    }
    
    for _ in range(num_rounds):
        action1 = predict_action(strategy1)  # You need to define how to predict actions based on your model
        action2 = predict_action(strategy2)
        
        # Update scores based on actions
        result = rewards[(action1, action2)]
        score1 += result[0]
        score2 += result[1]
    
    return score1, score2


In [39]:
def run_simulation(strategies, num_games, num_rounds):
    """
    Runs a Monte Carlo simulation where each strategy competes against every other.
    
    :param strategies: List of strategies/models.
    :param num_games: Number of games each pair of strategies will play.
    :param num_rounds: Number of rounds in each game.
    """
    results = {}  # Store the total score for each strategy
    
    for i, strategy1 in enumerate(strategies):
        for j, strategy2 in enumerate(strategies):
            total_score1, total_score2 = 0, 0
            
            # Play num_games games between strategy1 and strategy2
            for _ in range(num_games):
                score1, score2 = simulate_game(strategy1, strategy2, num_rounds)
                total_score1 += score1
                total_score2 += score2
            
            # Update the results
            results[(i, j)] = (total_score1, total_score2)
    
    return results


In [1]:
import itertools
import random

# Strategy functions modified to accept a history of moves
def always_trust(history):
    return "Trust"

def always_cheat(history):
    return "Cheat"

def tit_for_tat(history):
    if not history or history[-1] == "Trust":
        return "Trust"
    return "Cheat"

def half_cheat(history):
    recent_moves = history[-4:]
    if recent_moves.count("Cheat") >= 2:
        return "Cheat"
    return "Trust"

def alternate_cheat(history):
    if len(history) % 2 == 0:
        return "Cheat"
    return "Trust"

def random_choice(history):
    return random.choice(["Trust", "Cheat"])

# Adjust play_round to accept and update history
def play_round(strategy1, strategy2, history1, history2):
    action1 = strategy1(history1)
    action2 = strategy2(history2)
    return get_rewards(action1, action2), action1, action2

# Adjust play_game to maintain histories
def play_game(strategy1, strategy2, rounds):
    total_reward1, total_reward2 = 0, 0
    history1, history2 = [], []
    for _ in range(rounds):
        (reward1, reward2), action1, action2 = play_round(strategy1, strategy2, history1, history2)
        total_reward1 += reward1
        total_reward2 += reward2
        history1.append(action1)
        history2.append(action2)
    return total_reward1, total_reward2

# Run the simulation for a range of rounds
def run_simulation_for_rounds(strategies, games_per_matchup, round_options):
    for rounds_per_game in round_options:
        print(f"\nResults for {rounds_per_game} rounds per game:")
        simulation_results = run_simulation(strategies, games_per_matchup, rounds_per_game)

        # Print the results
        for matchup, scores in simulation_results.items():
            print(f"{matchup}: {scores}")

        # Rank strategies based on total rewards
        total_scores = {}
        for matchup, scores in simulation_results.items():
            total_scores[matchup[0]] = total_scores.get(matchup[0], 0) + scores[0]
            total_scores[matchup[1]] = total_scores.get(matchup[1], 0) + scores[1]

        sorted_scores = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
        print("\nStrategy Ranking (from highest to least rewards):")
        for rank, (strategy, score) in enumerate(sorted_scores, start=1):
            print(f"{rank}. {strategy}: {score}")

# Including all strategies in the simulation
strategies = [always_trust, always_cheat, tit_for_tat, half_cheat, alternate_cheat, random_choice]

# Simulation parameters
games_per_matchup = 100  # Number of games each strategy plays against each other
round_options = [5, 10, 15]  # Different numbers of rounds per game to simulate

# Run the simulation for various round options
run_simulation_for_rounds(strategies, games_per_matchup, round_options)


Results for 5 rounds per game:


NameError: name 'run_simulation' is not defined